# Put in your name below for merging the dataset later

In [ ]:
name = "L"

In [2]:
import xml
import cv2
import subprocess
import matplotlib.pyplot as plt
import sys
import os

from pathlib import Path


from ppadb.client import Client as AdbClient
import xml.dom.minidom
import re
import pandas as pd
import numpy as np
import seaborn as sns
from operator import itemgetter

import ipywidgets as widgets
from IPython.display import display

from functools import partial
from ipywidgets import interact, interactive, fixed, interact_manual
from PIL import Image

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from parse_xml import store_annotations

In [5]:
xml_annotations_path = Path("../xml_annotations/")
annotations_path = Path("../annotations/")
screenshots_path = Path("../screenshots/")
annotated_screenshots_path = Path("../annotated_screenshots/")

In [6]:
plt.style.use("seaborn-dark")
plt.rcParams["figure.figsize"] = 4.5, 8

In [7]:
def full_plotter(filename):
    image_file = screenshots_path/f"{filename}.png"
    csv_file = annotations_path/f"full_{filename}.csv"

    image = cv2.imread(str(image_file))

    annotation_df = pd.read_csv(csv_file)

    a_count = len(annotation_df.group.unique())

    color_palette = np.floor(np.array(sns.color_palette("Spectral", n_colors=a_count, desat=1)) * 255).astype(int).tolist()
    color_map = dict(zip(annotation_df.group.unique(), color_palette))

    get_pt1 = itemgetter("xmin", "ymin")
    get_pt2 = itemgetter("xmax", "ymax")

    boxed_image = image.copy()

    for i, data in annotation_df[annotation_df.widget_type != "layout"].iterrows():
        pt1 = get_pt1(data)
        pt2 = get_pt2(data)

        color_value = color_map[data.group]
        cv2.rectangle(boxed_image, pt1, pt2, color_value, 10)
        cv2.putText(boxed_image, data.classname, (pt1[0], pt1[1] - 5), cv2.FONT_HERSHEY_PLAIN, 4, (20, 30, 250), 3)
        
    plt.imshow(boxed_image)
    # cv2.imwrite(str(annotated_screenshots_path/f"{filename}.png"), boxed_image)

In [8]:
# full_plotter("L_screen_0000")

In [9]:
def plotter(filename):
    image_file = screenshots_path/f"{filename}.png"
    csv_file = annotations_path/f"{filename}.csv"

    image = cv2.imread(str(image_file))

    annotation_df = pd.read_csv(csv_file)

    a_count = len(annotation_df.group.unique())

    color_palette = np.floor(np.array(sns.color_palette("Spectral", n_colors=a_count, desat=1)) * 255).astype(int).tolist()
    color_map = dict(zip(annotation_df.group.unique(), color_palette))

    get_pt1 = itemgetter("xmin", "ymin")
    get_pt2 = itemgetter("xmax", "ymax")

    boxed_image = image.copy()

    for i, data in annotation_df[annotation_df.widget_type != "layout"].iterrows():
        pt1 = get_pt1(data)
        pt2 = get_pt2(data)

        color_value = color_map[data.group]
        cv2.rectangle(boxed_image, pt1, pt2, color_value, 10)
        cv2.putText(boxed_image, data.classname, (pt1[0], pt1[1] - 5), cv2.FONT_HERSHEY_PLAIN, 4, (20, 30, 250), 3)

    cv2.imwrite(str(annotated_screenshots_path/f"{filename}.png"), boxed_image)

# Get XML and screenshot from android

In [10]:
last_file_index = 0

if annotations_path.exists():
    existing_files = sorted(screenshots_path.glob(f"{name}_*.png"))

    if existing_files:
        last_file = existing_files[-1]
        last_file_index = int(re.findall(r"(\d+)", last_file.stem)[-1])

In [11]:
client = AdbClient(host="127.0.0.1", port=5037)
devices = client.devices()
for device in devices:
    print(device.serial)

192.168.0.24:5555


In [12]:
device = client.device("192.168.0.24:5555")

In [13]:
device.shell("echo 'Hello'")

'Hello\n'

> Important

## Run the cell below after changing every screen to get screenshot and xml

In [14]:
button = widgets.Button(description="Snap!")
label = widgets.Label(value=str(last_file_index))
image_disp = widgets.Image(format='png', width=300, height=400)
output = widgets.Output()

def snap_the_screen(b):
    global last_file_index
    
    filename = f"{name}_screen_{last_file_index:04}"
    
    # store_screenshot
    device.shell("screencap -p /sdcard/screen.png")
    device.pull("/sdcard/screen.png", (screenshots_path/f"{filename}.png"))

    # store full layout (Raw)
    device.shell("uiautomator dump")
    device.pull("/sdcard/window_dump.xml", (xml_annotations_path/f"full_{filename}.xml"))

    # store compressed (Relevant)
    device.shell("uiautomator dump --compressed")
    device.pull("/sdcard/window_dump.xml", (xml_annotations_path/f"{filename}.xml"))

    try:
        store_annotations(filename)
    except Exception as e:
        print(e)
    else:
        plotter(filename)

        last_file_index += 1
    
    with output:
        label.value = f"Total screenshots: {last_file_index}"
        with open((annotated_screenshots_path/f"{filename}.png"), "rb") as file:
            image = file.read()
            image_disp.value = image
            os.system("printf '\a'")
        
button.on_click(snap_the_screen)
widgets.VBox([button, label, image_disp, output])